In [11]:
pip install gdown

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
import os
import gdown
import shutil
import sys
import shutil
sys.path.append('../Module')
folder_path = '../Dataset'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn import set_config

from loader_csv import charger_csv
from data_to_csv import df_to_csv
from utils import generate_description
from currency import currency_cleaning
from naics_engineering import naicsEngineering

from Import_csv import import_csv
imported_csv_path = import_csv()

df = pd.read_csv(imported_csv_path)

Downloading...
From (original): https://drive.google.com/uc?id=1vdNaWcJgMzUnlaDVA4FuZRFYinO8UsPQ
From (redirected): https://drive.google.com/uc?id=1vdNaWcJgMzUnlaDVA4FuZRFYinO8UsPQ&confirm=t&uuid=f3223468-ae94-4abf-a76a-7649e93ede94
To: /home/utilisateur/Bureau/Brief Loan/FastAPI-MyHeadHurt/api/Model_pret/Dataset/SBAnational.csv
100%|██████████| 179M/179M [00:17<00:00, 10.2MB/s] 
/home/utilisateur/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3550: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[7187463009, 2172885003, 1950185008, 329725500..."
1,Name,object,14,0.00,779583,"[CREST WOODWORKS, MASTER DRY, Musicians Hut, I..."
2,City,object,30,0.00,32581,"[JEFFERSON, CARBONDALE, WASHINGTON, LOS ANGELE..."
3,State,object,14,0.00,51,"[TX, IL, OH, MN]"
4,Zip,int64,0,0.00,33611,"[15010, 83706, 85281, 19140, 98108]"
5,Bank,object,1559,0.17,5802,"[COUNTRY CLUB BANK, KEYBANK NATIONAL ASSOCIATI..."
6,BankState,object,1566,0.17,56,"[WI, NY, nan, IL, CA]"
7,NAICS,int64,0,0.00,1312,"[221111, 811111, 522310, 0, 233210]"
8,ApprovalDate,object,0,0.00,9859,"[28-May-03, 22-Apr-11, 24-Apr-06, 7-Nov-94, 30..."
9,ApprovalFY,object,0,0.00,70,"[2007, 2010, 2001, 1995]"


In [14]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [15]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)
df.NAICS.value_counts()

44-45    127251
81        72618
54        68170
31-33     68029
72        67600
23        66646
62        55366
42        48743
56        32685
48-49     22531
71        14640
53        13632
51        11379
52         9496
11         9005
61         6425
21         1851
22          663
55          257
92          229
Name: NAICS, dtype: int64

In [16]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)].astype(object)
df.NewExist.value_counts()

1.0    502830
2.0    193520
Name: NewExist, dtype: int64

In [17]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [18]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

df['RevLineCr'].value_counts()

N    484187
Y    195147
Name: RevLineCr, dtype: int64

In [19]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

df['LowDoc'].value_counts()

N    608823
Y     66203
Name: LowDoc, dtype: int64

In [20]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [21]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

df['Franchise'] = df['Franchise'].astype(object)
df['LowDoc'] = df['LowDoc'].astype(object)
df['RevLineCr'] = df['RevLineCr'].astype(object)
df['NewExist'] = df['NewExist'].astype(object)
df['ApprovalFY'] = df['ApprovalFY'].astype(object)
df['ApprovalDate'] = df['ApprovalDate'].astype(object)
df['NAICS'] = df['NAICS'].astype(object)
df['BankState'] = df['BankState'].astype(object)
df['Bank'] = df['Bank'].astype(object)
df['State'] = df['State'].astype(object)
df['City'] = df['City'].astype(object)


In [22]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [23]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [24]:
df.shape

(675026, 13)

In [25]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[81, 31-33, 72, 44-45]"
1,Term,object,0,0.0,377,"[288, 18, 241, 84, 60]"
2,NoEmp,object,0,0.0,499,"[2, 4, 3, 26]"
3,NewExist,object,0,0.0,2,"[2.0, 1.0]"
4,CreateJob,object,0,0.0,216,"[5, 0, 1]"
5,RetainedJob,object,0,0.0,323,"[1, 8, 0]"
6,UrbanRural,object,0,0.0,3,"[1, 0]"
7,RevLineCr,object,0,0.0,2,"[Y, N]"
8,LowDoc,object,0,0.0,2,[N]
9,GrAppv,int64,0,0.0,16880,"[51600, 410000, 184000, 40000, 10500]"


In [26]:
df_to_csv(df, folder_path, 'SBA_Cleaned_0.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_0.csv


In [27]:
df = pd.read_csv(imported_csv_path)

/home/utilisateur/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3550: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [28]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [29]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [30]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]

In [31]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [32]:
valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

In [33]:
valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

In [34]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [35]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

df['Franchise'] = df['Franchise'].astype(object)
df['LowDoc'] = df['LowDoc'].astype(object)
df['RevLineCr'] = df['RevLineCr'].astype(object)
df['NewExist'] = df['NewExist'].astype(object)
df['ApprovalFY'] = df['ApprovalFY'].astype(object)
df['ApprovalDate'] = df['ApprovalDate'].astype(object)
df['NAICS'] = df['NAICS'].astype(object)
df['BankState'] = df['BankState'].astype(object)
df['Bank'] = df['Bank'].astype(object)
df['State'] = df['State'].astype(object)
df['City'] = df['City'].astype(object)


In [36]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [37]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [38]:
df.shape

(460816, 13)

In [39]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[42, 44-45, 11]"
1,Term,int64,0,0.0,373,"[42, 84, 60, 36]"
2,NoEmp,int64,0,0.0,432,"[6, 1, 2, 3]"
3,NewExist,object,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,int64,0,0.0,192,[0]
5,RetainedJob,int64,0,0.0,276,"[3, 1, 0]"
6,UrbanRural,int64,0,0.0,3,"[1, 2, 0]"
7,RevLineCr,object,0,0.0,2,"[Y, N]"
8,LowDoc,object,0,0.0,2,[N]
9,GrAppv,int64,0,0.0,12824,"[100000, 40000, 409000, 5000]"


In [40]:
df_to_csv(df, folder_path, 'SBA_Cleaned_1.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_1.csv


In [41]:
df = pd.read_csv(imported_csv_path)

/home/utilisateur/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3550: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [42]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [43]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [44]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]

In [45]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [46]:
samples_bad_lol = df[(df['ChgOffDate'].notna())]
df.loc[samples_bad_lol.index, 'MIS_Status'] = samples_bad_lol['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')

In [47]:
samples_bad = df[(df['ChgOffPrinGr'].notna()) & ((df['ChgOffPrinGr'] != 0.0) | (df['ChgOffPrinGr'] > 0.0))]
df.loc[samples_bad.index, 'MIS_Status'] = samples_bad['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).fillna(0.0).astype(float).astype(int)

In [48]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

In [49]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

In [50]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [51]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [52]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [53]:
df.shape

(675026, 13)

In [54]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[62, 81, 54]"
1,Term,int64,0,0.0,377,"[32, 68, 29, 208, 87]"
2,NoEmp,int64,0,0.0,499,"[24, 84, 1, 7]"
3,NewExist,float64,0,0.0,2,"[2.0, 1.0]"
4,CreateJob,int64,0,0.0,216,"[1, 4, 8, 0, 2]"
5,RetainedJob,int64,0,0.0,323,"[0, 7, 16]"
6,UrbanRural,int64,0,0.0,3,"[1, 2]"
7,RevLineCr,object,0,0.0,2,"[Y, N]"
8,LowDoc,object,0,0.0,2,[N]
9,GrAppv,int64,0,0.0,16880,"[85000, 210000, 150000, 121500, 618000]"


In [55]:
df_to_csv(df, folder_path, 'SBA_Cleaned_2.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_2.csv


In [56]:
df = pd.read_csv(imported_csv_path)

/home/utilisateur/.local/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3550: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [57]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[9452373007, 1709005004, 1031225006, 712955300..."
1,Name,object,14,0.00,779583,"[SPARKLE OPTICS CORPORATION, INGREDIENTS SPECI..."
2,City,object,30,0.00,32581,"[EL PASO, GRASS VALLEY, TOLLAND, STEVENS POINT..."
3,State,object,14,0.00,51,"[KS, CA, MA, RI, CT]"
4,Zip,int64,0,0.00,33611,"[51250, 44319, 66502, 95340, 33013]"
5,Bank,object,1559,0.17,5802,"[CALIFORNIA UNITED BANK, WISCONSIN BUS. DEVEL ..."
6,BankState,object,1566,0.17,56,"[RI, DE, KY, IL]"
7,NAICS,int64,0,0.00,1312,"[423920, 0, 337110, 448310, 111998]"
8,ApprovalDate,object,0,0.00,9859,"[29-May-98, 1-May-95, 27-Oct-89, 5-Jun-12, 23-..."
9,ApprovalFY,object,0,0.00,70,"[1999, 1991, 2000, 2008, 2004]"


In [58]:
df = df.drop_duplicates(subset='LoanNr_ChkDgt')

In [59]:
df['NewExist'] = df['NewExist'].isin({1.0, 2.0}).astype(object)

In [60]:
is_valide = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].isin(is_valide).astype(object)

In [61]:
is_valide = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].isin(is_valide).astype(object)

In [63]:
df = df.dropna(subset=['NAICS'])
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [64]:
mapping = {0: 'unknown', 1: 'urban', 2: 'rural'}
df['UrbanRural'] = df['UrbanRural'].map(mapping).astype(object)

In [65]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1).astype(object)
df = df.drop('FranchiseCode', axis=1)

In [66]:
df['ApprovalDate'] = pd.to_datetime(df['ApprovalDate'], format='%d-%b-%y').dt.strftime('%d-%m-%y').astype(object)

In [67]:
df['ApprovalDate'].unique()

array(['28-02-97', '07-02-06', '25-03-97', ..., '25-02-97', '26-02-97',
       '27-02-97'], dtype=object)

In [68]:
df['ApprovalFY'] = df['ApprovalFY'].astype(str).apply(lambda x: '20' + x[-2:] if int(x[-2:]) >= 0 and int(x[-2:]) <= 27 else '19' + x[-2:]).astype(object)
df['ApprovalFY'].unique()

array(['1997', '2006', '1998', '1999', '2000', '2001', '2003', '2004',
       '2005', '1984', '2007', '1987', '2008', '1988', '2009', '1989',
       '1990', '2010', '1991', '2011', '1992', '2002', '2012', '1993',
       '2013', '1994', '2014', '1995', '1996'], dtype=object)

In [69]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [70]:
samples_bad = df[(df['ChgOffPrinGr'].notna()) & (df['ChgOffPrinGr'] != 0.0)]
df.loc[samples_bad.index, 'MIS_Status'] = samples_bad['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).fillna(0.0).astype(float).astype(int)

In [71]:
samples_bad_lol = df[df['ChgOffDate'].notna()]
df.loc[samples_bad_lol.index, 'MIS_Status'] = samples_bad_lol['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')

In [72]:
df['MIS_Status'].value_counts()

P I F     550334
CHGOFF    145335
Name: MIS_Status, dtype: int64

In [73]:
df['Approve'] = df['MIS_Status'].apply(lambda x: 1 if x == 'P I F' else 0).astype(bool)
df.drop(['MIS_Status'], axis=1, inplace=True)

In [74]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Zip', 'LoanNr_ChkDgt']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df = df.dropna()

In [75]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,Name,object,0,0.0,611131,"[PARKS AUTOMOTIVE,INC., FLORIDA SHORTENING COR..."
1,City,object,0,0.0,28860,"[Miamisburg, OKLAHOMA CITY, EVANSVILLE, IRVING..."
2,State,object,0,0.0,51,"[CA, WA, NH, NM, ME]"
3,Bank,object,0,0.0,5221,"[CAPITAL ONE NATL ASSOC, THE FIRST STATE BANK,..."
4,BankState,object,0,0.0,55,"[OH, CO, CA, WA]"
5,NAICS,object,0,0.0,20,"[54, 81, 42]"
6,ApprovalDate,object,0,0.0,7367,"[13-08-92, 04-01-06, 13-07-09, 08-09-09, 25-09..."
7,ApprovalFY,object,0,0.0,29,"[2002, 2006, 2003, 2009, 1996]"
8,Term,int64,0,0.0,378,"[60, 240, 180]"
9,NoEmp,int64,0,0.0,502,"[1, 8, 9, 2]"


In [76]:
df.shape

(696572, 20)

In [77]:
df_to_csv(df, folder_path, 'SBA_Cleaned_Pure.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_Pure.csv
